In [1]:
import pandas as pd

In [28]:
DATADIR = '~/Data/sports/volleyball/'
stopwords = set(pd.read_csv('~/Data/general-knowledge/english_stopwords_nltk.txt', header=None)[0])

In [54]:
def normalise(s): 
    return " ".join([v for v in "".join([w for w in s.replace("/"," ").replace("-", " ").lower() 
                                if (w.isalnum() or w == ' ')]).split() if v not in stopwords])

def find_abbreviatons(s):
    return True if set(normalise(s).split()) & abbrs_volleyball else False

def find_countries(s):
    return True if len(set(normalise(s).split()) & cntrs_volleyball) == 2 else False

In [58]:
tkt_events = pd.read_csv('~/Data/ticketek-shows/all-tkt-events-11092017.csv.gz', encoding='latin-1', sep='\t')

In [59]:
tkt_events.head()

,event_id,description
0,1,unknown - - - - - - -
1,2,unknown - - - - - - -
2,3,r carlos nakai - iwaki auditorium r. carl...
3,4,karen schaupp - iwaki auditorium karin s...
4,5,unknown - - - - - - -


In [49]:
comps_volleyball = [normalise(comp) for comp in pd.read_csv(DATADIR + 'comps_volleyball_11092017.txt', header=None)[0].tolist()]
abbrs_volleyball = set(pd.read_csv(DATADIR + 'abbrs_volleyball_11092017.txt', header=None)[0])
cntrs_volleyball = set(pd.read_csv(DATADIR + 'cntrs_volleyball_11092017.txt', header=None)[0])

In [50]:
abbrs = {"championships": ["champs"]}
country_alt = {"united states": ["usa", "united states of america", "us"],
              "russia": ["russian federation"],
              "chinese taipei": ["taiwan"],
              "macedonia": ["fyrom"],
              "netherlands": ["holland"]}

In [51]:
cntrs_volleyball

{'argentina',
 'australia',
 'austria',
 'belgium',
 'brazil',
 'bulgaria',
 'canada',
 'china',
 'chinese taipei',
 'czech republic',
 'egypt',
 'estonia',
 'finland',
 'france',
 'germany',
 'greece',
 'iran',
 'italy',
 'japan',
 'kazakhstan',
 'mexico',
 'montenegro',
 'netherlands',
 'poland',
 'portugal',
 'qatar',
 'russia',
 'serbia',
 'slovakia',
 'slovenia',
 'south korea',
 'spain',
 'tunisia',
 'turkey',
 'united states',
 'venezuela'}

In [52]:
abbrs_volleyball

{'avc', 'avl', 'cev', 'cism', 'fivb', 'norceca', 'qbvt', 'sabvs', 'wch'}

In [55]:
tkt_events.loc[tkt_events.description.apply(find_abbreviatons) | tkt_events.description.apply(find_countries),:]

,_event_id,perf_y,description
399,1651,2002,wallabies v france test - stadium australia ...
591,304,2002,evolution - ais arena canberra evoluti...
1264,1367,1999,"1999 intercontinental cup, baseball - blacktow..."
1267,1370,1999,"1999 intercontinental cup, baseball - blacktow..."
2814,5673,2002,evolution - history of circus & gymnastics - b...
3521,5000,1999,socceroos v brazil - stadium australia lg so...
3597,9459,2001,national museum of australia opening day conce...
3736,5041,2002,wallabies v france test - stadium australia ...
4069,8580,2000,"womens soccer at sfs, bruce, breakers the ma..."
4227,8609,1999,canberra cosmos soccer - bruce stadium canberr...
